15分でできる日本語Word2Vec https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

In [8]:
import json
import urllib.request
from janome.tokenizer import Tokenizer
from gensim.models import word2vec

In [1]:
# zipファイルダウンロード
url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = 'data/794_ruby_4237.zip'
urllib.request.urlretrieve(url, zip)

# ダウンロードしたzipの解凍
import zipfile
with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    # 解凍後のファイルからデータ読み込み
    for myfile in myzip.infolist():
        # 解凍後ファイル名取得
        filename = myfile.filename
        # ファイルオープン時にencodingを指定してsjisの変換をする
        with open(filename, encoding='sjis') as file:
            text = file.read()

# ファイル整形
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。




In [2]:
# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

一
する
目
さめる
女
隣
じいさん
話
始める
いる


sentences=None, 
corpus_file=None, 
vector_size=100, 
alpha=0.025, 
window=5, 
min_count=5, 
max_vocab_size=None, 
sample=0.001, 
seed=1, 
workers=3, 
min_alpha=0.0001, 
sg=0, 
hs=0, 
negative=5, 
ns_exponent=0.75, 
cbow_mean=1, 
hashfxn=<built-in function hash>, 
epochs=5, 
null_word=0, 
trim_rule=None, 
sorted_vocab=1, 
batch_words=10000, 
compute_loss=False, 
callbacks=(), 
comment=None, 
max_final_vocab=None)

In [3]:
# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
# print(help(word2vec.Word2Vec()))
model = word2vec.Word2Vec(word_list, vector_size=100,min_count=5,window=5,epochs=100)

C:\Users\haruk\AppData\Local\Programs\Python\Python39\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [4]:
print(model.__dict__['wv']['世間'])

[-0.02342418  0.7995041  -0.06785399  0.02257429 -0.4907826  -0.5104657
 -0.14073755  0.4452458  -0.23098688  0.17104453  0.42721802 -0.46468538
  0.12858783  0.52194965  0.2275441  -0.12669651 -0.06366496 -0.0021622
  0.22506769 -0.80989295 -0.9734439   0.4786812   0.11691452 -1.1231492
  0.2593213   0.04639455 -0.56368816 -0.35878667 -0.19768499 -0.82413137
  0.43515825 -0.22792846 -0.09085202 -0.2186421  -0.31527388  0.24011101
 -0.53099126 -0.26208246 -0.26015916  0.01928342  0.46294722 -0.12994617
 -0.73061335 -0.6540881  -0.39690083  0.6186091  -0.10938062 -0.565133
  0.5356147  -0.53265846  0.6592306  -0.38061428  0.6069152  -0.40520078
  0.08645136 -0.6913601   0.43534136 -0.06651203 -0.46202615  0.5396672
 -0.76363456  0.17021827 -0.63488066 -0.5545133  -0.8611824   0.30002668
 -0.12758845 -0.45502862  0.1931637   0.2086118  -0.39836025 -0.5607809
 -0.3781595  -0.06220999  0.45573375 -0.01196804  0.10914601  0.42883742
  0.208486    0.29640514 -0.7889408   0.16478486 -0.298284

In [5]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['世間']) 
for item in ret:
    print(item[0], item[1])

聞こえる 0.5507597327232361
喝采 0.5462894439697266
自己 0.5396975874900818
賛成 0.5204139947891235
社会 0.4910232722759247
外国 0.4857619106769562
有す 0.46897920966148376
決心 0.4653388261795044
今日 0.45626208186149597
彼ら 0.45155787467956543


In [9]:
path = 'data/recipe_chefgohan.json'
df = json.load(open(path, 'r', encoding='utf-8'))
# print(df)
text = ''
for _,r in df['recipes'].items():
    # print(r['title'])
    start = r['title']+'を作ります。'
    end = r['title']+'が完成しました。'
    instructions = ''.join(r['instructions']).replace('\n','')
    text += start + instructions + end + '\n'
print(text[:100])

さつま芋のもちもちチーズ羊羹を作ります。牛乳と生クリームを合わせ、棒寒天を加えて煮溶かす。下準備したさつまいもとクリームチーズを合わせてよく混ぜてSTEP1をこしながら加える。さらに小麦粉とラム酒を加


In [10]:
# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

さつま芋
もち
もち
チーズ
羊羹
作る


In [11]:
model = word2vec.Word2Vec(word_list, vector_size=300,min_count=5,window=5,epochs=100)

In [12]:
print(model.__dict__['wv']['肉'])

[-0.5986756   0.11120176  2.483063   -0.99792653  0.94078827 -2.1803505
  1.7289515  -0.78446496 -0.6961496  -0.6034527  -1.9562347   1.8921211
 -1.2863146  -0.44978943 -0.45571327 -1.2739933   0.18854299 -0.7997567
 -0.35115007  0.15794533  0.6661978   0.95137966  0.2010077  -0.5620384
 -0.17434746 -0.55147153 -0.9045373  -0.4674271  -0.0425252  -1.3209642
 -1.2287056   0.23286985  0.54537106  0.3729948   0.94298756 -0.49929747
  0.21364297  1.2980967   1.555055   -1.496617    0.00643672 -0.78650355
 -0.62216556  1.9311621   0.67826766  0.2670711   0.8559301  -0.13909958
  0.6537466  -0.7627413   0.51864386 -0.09093025 -0.18669231 -1.9206095
  0.11453193  1.872001    1.2213339  -0.2928359   0.8865545  -0.09018312
  0.7639236   0.44107923  0.6790276   0.42932513  0.39496636  0.648185
 -0.5341494   1.759587   -0.8662903  -0.9017549  -1.0480204  -0.67416567
  0.113922   -0.53868866  0.02932351  1.366291   -0.08127295  0.8078089
  0.9044685   0.26821086 -0.24697204  0.15461186 -1.2659596 

In [13]:
ret = model.wv.most_similar(positive=['焼く']) 
for item in ret:
    print(item[0], item[1])

焼ける 0.49773409962654114
キツネ 0.47269976139068604
トースター 0.45389020442962646
焼き上げる 0.45307379961013794
返す 0.4320129156112671
面 0.3933850824832916
取り出し 0.39319199323654175
焼き 0.3683605492115021
押す 0.35697951912879944
両面 0.3562738597393036


In [14]:
results = model.wv.most_similar(positive=['焼く','水'], topn=5)
for result in results:
    print(result[0])

取り出し
キツネ
冷水
加工
テフロン


In [15]:
results = model.wv.most_similar(positive=['煮る'], negative=['水'], topn=5)
for result in results:
    print(result[0])

煮込む
絹
タイ
俺
チャーハン


In [16]:
results = model.wv.most_similar(positive=['煮る'], negative=['焼く'], topn=5)
for result in results:
    print(result[0])

顆粒
だし
A
ペースト
コンソメ


In [17]:
results = model.wv.most_similar(positive=['肉','たまねぎ'], topn=5)
for result in results:
    print(result[0])

挽き肉
きゅうり
ロース
玉ねぎ
えび
